In [3]:
# Importing the libraries 
import pandas as pd
import numpy as np
import random
import pickle
import re
import regex  # for better, more capabale regex api
import os
import zipfile
import more_itertools
from itertools import chain
import datetime
import time
from statsmodels.stats.proportion import proportion_confint
# active labeler related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline
from superintendent import ClassLabeller
from IPython.display import display, Markdown

pd.set_option('display.max_colwidth', None)  # so we can peak at data and spot verify

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
print('done')

done


In [4]:
# Set up columns to keep, fields, locations for writing
rootpath = "/hdd/work/d4ad_standardization/"
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"
interimpath = "D4AD_Standardization/data/interim/"

content_is = "standardized_descriptions_and_degree_funding_type"

print('done')

done


In [5]:

filepath = content_is + ".csv" # builds off of notebook 6.0 work

SKIP_THIS = True # helps me be able to run all and not worry about pulling things
# I already know I have on disk

df = pd.read_csv(rootpath + interimpath + filepath)
print('done2')

done2


In [6]:
pd.set_option('display.max_rows', False)

the_df = df #df.sample(n=10000, random_state=42)

In [7]:
training_context_regex =\
    """
    ((\w+\W+){0,8}        # first 8 or so words before
     (?P<job_search>job[\s\b.].*?search|assist[\w\s\b\.].*?employ\w*?\b) # help w/ job search
     (\W+\w+){0,8})       # and last 8 or so word after
    """

wioa_indices = the_df['IS_WIOA'] == True
the_df['has_minimum_six_months_search'] =\
    False
the_df.loc[wioa_indices, 'minimum_six_months_search'] =\
    True

# Extract job search context...
job_search_length_mention =\
    the_df.loc[wioa_indices, 'DESCRIPTION']\
          .str\
          .extractall(pat=training_context_regex, flags=re.I|re.VERBOSE)[0]

the_df['additional_job_search_duration_mention'] =\
    None

# ... all mentions follow <numeric reference> ... <time duration> format
# e.g. three hours per day, 4-week job readiness, Four week job search, etc.

# We'll take the simpler approach of treating the duration as the anchor
# and grab those statements that have a numeric 0, 1, 2, 3 or 4 words before it

numbers = "one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen"
durations = "minute|day|week|month|year"
duration_regex =\
   f"""
    (?P<numeric>\d|{numbers})       # numeric reference ...
    (?P<modifer>.*)                 # followed by content that might modify ...
    (?P<base_duration>{durations})     # ... the base duration
    """

job_search_length_mention =\
    job_search_length_mention.str.extract(
        duration_regex,
        flags=re.I|re.VERBOSE
    ).replace('-', '')\
    .replace('week', 'weeks')\
    .dropna()\
    .droplevel('match')  # drop weird 

the_df.loc[job_search_length_mention.index,
           'additional_job_search_duration_mention'] =\
               job_search_length_mention['numeric'] +\
               job_search_length_mention['modifer'] +\
               job_search_length_mention['base_duration']
print('done')



done


In [8]:
# 4)
# Now we write out the verfied results
# ... finally we can write this out as our first complete lookup table
# for the NAME field
write_out = the_df

print(
    "We're writing ...",
    write_out.columns
)

content_is = "with_job_search_durations"

# shuffe the rows to better remove temporal baises
write_out =\
    write_out.sample(frac=1, random_state=42, axis=0).reset_index(drop=True)

write_out.to_csv(rootpath + interimpath + content_is + ".csv",
                index = False,
                chunksize = 10000)

write_out.to_excel(rootpath + processedpath + content_is + ".xls",
            sheet_name="Standardized Descriptions",
            index=False)
print('done')

We're writing ... Index(['IS_WIOA', 'Mentioned_Certificate', 'Mentioned_Associates',
       'STANDARDIZED_DESCRIPTION', 'STANDARDIZED_FEATURESDESCRIPTION',
       'CLEANED_STANDARDIZED_NAME_1', 'STANDARDIZEDNAME', 'STANDARDIZEDNAME_1',
       'DESCRIPTION', 'FEATURESDESCRIPTION', 'NAME_1', 'NAME',
       'has_minimum_six_months_search', 'minimum_six_months_search',
       'additional_job_search_duration_mention'],
      dtype='object')
done
